In [153]:
import unidecode
import numpy as np
import pandas as pd
from pathlib import Path
import os; os.chdir('..')
PATH = Path('data/raw/inen_series')

In [154]:
!ls {PATH}

datos10.xlsx  datos2.xlsx  datos4.xlsx	datos6.xlsx  datos8.xlsx
datos1.xlsx   datos3.xlsx  datos5.xlsx	datos7.xlsx  datos9.xlsx


In [187]:
def read_data(path):
    df = pd.read_excel(path, skiprows=2)#, skip_footer=9)
    df.columns = [unidecode.unidecode(e.lower().strip()) for e in df.columns]
    n = df[df.ambito == 'Fuente(s):'].reset_index()['index'].iat[0]
    df = df.applymap(lambda x: np.nan if x == '-' or x == '...' else x).iloc[:n-1]
    df.ambito = df.ambito.apply(lambda x: unidecode.unidecode(str(x).strip()))
    df.indicador = df.indicador.apply(lambda x: unidecode.unidecode(x.lower().strip().replace(' ', '_')))
    return df

def read_all(path):
    dfs = [read_data(e) for e in path.iterdir() if e.suffix == '.xlsx']
    df = pd.concat(dfs).loc[:, dfs[0].columns]
    return df

In [188]:
df = read_all(PATH)
df.head()

,ambito,indicador,unidad,1960,1961,1962,1963,1964,1965,1966,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Amazonas,precio_promedio_pagado_al_productor_de_fibra_d...,Soles/kg.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amazonas,precio_promedio_pagado_al_productor_de_fibra_d...,Soles/kg.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amazonas,produccion_de_madera_aserrada,Miles de metros cúbicos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11463.00,8900.0,10290.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amazonas,produccion_de_parquet,Miles de metros cúbicos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Amazonas,produccion_de_carbon,Miles de metros cúbicos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
df.to_csv('data/processed/inei_series.csv', index=False)